In [4]:
import pandas as pd
import numpy as np

In [5]:
movies = pd.read_csv('datasets/movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
import re

In [7]:
def clean_title(title):
    return re.sub("[^a-zA-Z0-9 ]","",title)

In [8]:
movies["clean title"] = movies["title"].apply(clean_title)

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
vectorizer = TfidfVectorizer(ngram_range=(1,2))
tfidf = vectorizer.fit_transform(movies["clean title"])

In [11]:
from sklearn.metrics.pairwise import cosine_similarity

In [12]:
def search(title):
    title = clean_title(title)
    query_vec = vectorizer.transform([title])
    similarity = cosine_similarity(query_vec, tfidf).flatten()
    indices = np.argpartition(similarity, -5)[-5:]
    results = movies.iloc[indices][::-1]
    return results

In [13]:
search('Toy Story 1995')

,movieId,title,genres,clean title
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 1995
3021,3114,Toy Story 2 (1999),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 2 1999
59767,201588,Toy Story 4 (2019),Adventure|Animation|Children|Comedy,Toy Story 4 2019
14813,78499,Toy Story 3 (2010),Adventure|Animation|Children|Comedy|Fantasy|IMAX,Toy Story 3 2010
20497,106022,Toy Story of Terror (2013),Animation|Children|Comedy,Toy Story of Terror 2013


In [14]:
import ipywidgets as widgets
from IPython.display import display

In [15]:
def on_type(data):
    with movie_list:
        movie_list.clear_output()
        title = data["new"]
        if len(title)>5:
            display(search(title))

In [16]:
import ipywidgets as widgets
from IPython.display import display

# Create the Text widget
movie_input = widgets.Text(
    value="Toy Story",
    description="Movie Title:",
    disabled=False
)

# Create the Output widget
movie_list = widgets.Output()

# Function to handle changes in Text widget
def on_type(data):
    with movie_list:
        movie_list.clear_output()
        title = data["new"]
        if len(title)>5:
            display(search(title))
# Attach the handler to the Text widget
movie_input.observe(on_type, names='value')

# Display the widgets
display(movie_input)
display(movie_list)


Text(value='Toy Story', description='Movie Title:')

Output()

In [17]:
ratings = pd.read_csv('datasets/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [23]:
movie_id = 1

In [26]:
similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"]> 4)]["userId"].unique()
similar_users

array([    36,     75,     86, ..., 162527, 162530, 162533], dtype=int64)

In [39]:
similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]
similar_user_recs

5101            1
5105           34
5111          110
5114          150
5127          260
            ...  
24998854    60069
24998861    67997
24998876    78499
24998884    81591
24998888    88129
Name: movieId, Length: 1358326, dtype: int64

In [40]:
similar_user_recs = similar_user_recs.value_counts() / len(similar_users)
similar_user_recs = similar_user_recs[similar_user_recs > .1]

In [41]:
similar_user_recs

movieId
1        1.000000
318      0.445607
260      0.403770
356      0.370215
296      0.367295
           ...   
953      0.103053
551      0.101195
1222     0.100876
745      0.100345
48780    0.100186
Name: count, Length: 113, dtype: float64

In [44]:
all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]

In [47]:
all_users_recs = all_users["movieId"].value_counts()/ len(all_users["userId"].unique())

In [48]:
all_users_recs

movieId
318      0.342220
296      0.284674
2571     0.244033
356      0.235266
593      0.225909
           ...   
551      0.040918
50872    0.039111
745      0.037031
78499    0.035131
2355     0.025091
Name: count, Length: 113, dtype: float64

In [49]:
rec_precentetages = pd.concat([similar_user_recs, all_users_recs], axis=1)
rec_precentetages.columns = ["similar", "all"]

In [50]:
rec_precentetages

,similar,all
movieId,,
1,1.000000,0.124728
318,0.445607,0.342220
260,0.403770,0.222207
356,0.370215,0.235266
296,0.367295,0.284674
...,...,...
953,0.103053,0.045792
551,0.101195,0.040918
1222,0.100876,0.066877


In [51]:
rec_precentetages["score"] = rec_precentetages["similar"]/ rec_precentetages["all"] 

In [52]:
rec_precentetages.sort_values("score", ascending=False, inplace=True)

In [53]:
rec_precentetages

,similar,all,score
movieId,,,
1,1.000000,0.124728,8.017414
3114,0.280648,0.053706,5.225654
2355,0.110539,0.025091,4.405452
78499,0.152960,0.035131,4.354038
4886,0.235147,0.070811,3.320783
...,...,...,...
2858,0.216724,0.167634,1.292845
296,0.367295,0.284674,1.290232
79132,0.166817,0.131384,1.269693


In [54]:
rec_precentetages.head(10).merge(movies, left_index = True, right_on = "movieId")

,similar,all,score,movieId,title,genres,clean title
0,1.000000,0.124728,8.017414,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 1995
3021,0.280648,0.053706,5.225654,3114,Toy Story 2 (1999),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 2 1999
2264,0.110539,0.025091,4.405452,2355,"Bug's Life, A (1998)",Adventure|Animation|Children|Comedy,Bugs Life A 1998
14813,0.152960,0.035131,4.354038,78499,Toy Story 3 (2010),Adventure|Animation|Children|Comedy|Fantasy|IMAX,Toy Story 3 2010
4780,0.235147,0.070811,3.320783,4886,"Monsters, Inc. (2001)",Adventure|Animation|Children|Comedy|Fantasy,Monsters Inc 2001
580,0.216618,0.067513,3.208539,588,Aladdin (1992),Adventure|Animation|Children|Comedy|Musical,Aladdin 1992
6258,0.228139,0.072268,3.156862,6377,Finding Nemo (2003),Adventure|Animation|Children|Comedy,Finding Nemo 2003
587,0.179400,0.059977,2.991150,595,Beauty and the Beast (1991),Animation|Children|Fantasy|Musical|Romance|IMAX,Beauty and the Beast 1991
8246,0.203504,0.068453,2.972889,8961,"Incredibles, The (2004)",Action|Adventure|Animation|Children|Comedy,Incredibles The 2004
359,0.253411,0.085764,2.954762,364,"Lion King, The (1994)",Adventure|Animation|Children|Drama|Musical|IMAX,Lion King The 1994


In [65]:
def find_similar_movies(movie_id):
    similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"]>4)]["userId"].unique()
    similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"]>4)]["movieId"]
    
    similar_user_recs = similar_user_recs.value_counts() / len(similar_users)
    similar_user_recs = similar_user_recs[similar_user_recs > .10]
    
    all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"]>4)]
    all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())
    
    rec_precentetages = pd.concat([similar_user_recs, all_users_recs], axis=1)
    rec_precentetages.columns = ["similar", "all"]

    rec_precentetages["score"] = rec_precentetages["similar"] / rec_precentetages["all"]
    
    rec_precentetages.sort_values("score", ascending=False, inplace=True)
    return rec_precentetages.head(10).merge(movies, left_index = True, right_on = "movieId")[["score","title","genres"]]

In [74]:
title = "Harry Potter and the Prisoner of Azkaban (2004)"
results = search(title)
movie_id = results.iloc[0]["movieId"]
find_similar_movies(movie_id)

,score,title,genres
7742,20.652079,Harry Potter and the Prisoner of Azkaban (2004),Adventure|Fantasy|IMAX
4790,12.037642,Harry Potter and the Sorcerer's Stone (a.k.a. ...,Adventure|Children|Fantasy
6416,4.923686,Pirates of the Caribbean: The Curse of the Bla...,Action|Adventure|Comedy|Fantasy
12324,4.568910,Iron Man (2008),Action|Adventure|Sci-Fi
11361,4.538842,Ratatouille (2007),Animation|Children|Drama
14813,4.371778,Toy Story 3 (2010),Adventure|Animation|Children|Comedy|Fantasy|IMAX
6258,4.322308,Finding Nemo (2003),Adventure|Animation|Children|Comedy
4780,4.191061,"Monsters, Inc. (2001)",Adventure|Animation|Children|Comedy|Fantasy
8246,4.179564,"Incredibles, The (2004)",Action|Adventure|Animation|Children|Comedy
13362,3.689451,Up (2009),Adventure|Animation|Children|Drama
